# **Setting up the environment**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from pathlib import Path

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

2638744/2638744 [==============================] - 0s 0us/step


In [4]:
np.random.seed(42)

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [5]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [6]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [7]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [8]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [9]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [10]:
tf.random.set_seed(42) 
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [11]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

# **Bidirectional RNNs - Sequence to Sequence Model**

In [12]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

In [13]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

In [14]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=15,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/15
3125/3125 [==============================] - 104s 28ms/step - loss: 2.7680 - accuracy: 0.4507 - val_loss: 1.9886 - val_accuracy: 0.5548
Epoch 2/15
3125/3125 [==============================] - 60s 19ms/step - loss: 1.6996 - accuracy: 0.6018 - val_loss: 1.5497 - val_accuracy: 0.6308
Epoch 3/15
3125/3125 [==============================] - 58s 19ms/step - loss: 1.3489 - accuracy: 0.6669 - val_loss: 1.3757 - val_accuracy: 0.6653
Epoch 4/15
3125/3125 [==============================] - 60s 19ms/step - loss: 1.1504 - accuracy: 0.7054 - val_loss: 1.2942 - val_accuracy: 0.6802
Epoch 5/15
3125/3125 [==============================] - 59s 19ms/step - loss: 1.0056 - accuracy: 0.7349 - val_loss: 1.2680 - val_accuracy: 0.6860
Epoch 6/15
3125/3125 [==============================] - 59s 19ms/step - loss: 0.8887 - accuracy: 0.7601 - val_loss: 1.2624 - val_accuracy: 0.6890
Epoch 7/15
3125/3125 [==============================] - 59s 19ms/step - loss: 0.7888 - accuracy: 0.7823 - val_loss: 1.2762 

# **Greedy Decoding**

In [15]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input 
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [16]:
translate("I like soccer")

1/1 [==============================] - 0s 23ms/step


'me gusta el fútbol'

In [17]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 23ms/step


'me gusta la playa a la playa estoy [UNK] a la playa'

In [18]:
model.save("bidirecRNN" , save_format="tf")

# **Beam Search**

In [19]:
def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # encoder input
    X_dec = np.array(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]
    
    # extra code – displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = np.array([sentence_en])  # encoder input
            X_dec = np.array(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # extra code – displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [20]:
sentence_en = "I love cats and dogs"
translate(sentence_en)

1/1 [==============================] - 0s 23ms/step


'me [UNK] las perros y chicos'

In [21]:
beam_search(sentence_en, beam_width=3, verbose=True)

1/1 [==============================] - 0s 23ms/step
Top first words: [(-0.26484933, 'me'), (-1.6776025, '[UNK]'), (-3.9730356, 'odio')]
1/1 [==============================] - 0s 25ms/step
Top translations so far: [(-0.6323493, 'me [UNK]'), (-1.5653938, 'me gustan'), (-2.6096168, '[UNK] los')]
1/1 [==============================] - 0s 25ms/step
Top translations so far: [(-1.2610011, 'me [UNK] las'), (-1.6146307, 'me gustan los'), (-1.6942544, 'me [UNK] los')]
1/1 [==============================] - 0s 26ms/step
Top translations so far: [(-1.7753377, 'me gustan los perros'), (-1.8302231, 'me [UNK] los gatos'), (-2.049029, 'me [UNK] las perros')]
1/1 [==============================] - 0s 24ms/step
Top translations so far: [(-1.775629, 'me gustan los perros y'), (-1.830763, 'me [UNK] los gatos y'), (-2.0493615, 'me [UNK] las perros y')]
1/1 [==============================] - 0s 24ms/step
Top translations so far: [(-1.8949953, 'me [UNK] los gatos y perros'), (-2.1989937, 'me gustan los perro

'me [UNK] los gatos y perros'